# Losses

## Initial imports

In [71]:
import numpy as np
import pandas as pd
import torch


# increase displayed columns in jupyter notebook
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 300)

# Pytorch implementation
* reduction="none" in binary_cross_entropy_with_logits gives same result for given testing usecase as authors implementation https://pytorch.org/docs/stable/generated/torch.nn.functional.binary_cross_entropy_with_logits.html
  * hovewer keras BCE uses auto reduction by default https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy which changes according to usecase : so wtf? https://www.tensorflow.org/api_docs/python/tf/keras/losses/Reduction
* keras also return flattened BCE result
* keras needs loss per sample and it averages it in the backend: https://keras.io/api/losses/

In [83]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_widedeep.wdtypes import *


def _predict_ziln(preds: Tensor) -> Tensor:
    """Calculates predicted mean of zero inflated lognormal logits.
    Adjusted implementaion of `code
    <https://github.com/google/lifetime_value/blob/master/lifetime_value/zero_inflated_lognormal.py>`
    Arguments:
        preds: [batch_size, 3] tensor of logits.
    Returns:
        ziln_preds: [batch_size, 1] tensor of predicted mean.
    """
    positive_probs = torch.sigmoid(preds[..., :1])
    loc = preds[..., 1:2]
    scale = F.softplus(preds[..., 2:])
    ziln_preds = (
        positive_probs *
        torch.exp(loc + 0.5 * torch.square(scale)))
    return ziln_preds


class ZILNLoss(nn.Module):
    r"""Adjusted implementation of the `Zero Inflated LogNormal loss
    <https://arxiv.org/pdf/1912.07753.pdf>` and its `code
    <https://github.com/google/lifetime_value/blob/master/lifetime_value/zero_inflated_lognormal.py>`
    """

    def __init__(self):
        super().__init__()

    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        r"""
        Parameters
        ----------
        input: Tensor
            input tensor with predictions (not probabilities)
        target: Tensor
            target tensor with the actual classes

        Examples
        --------
        >>> import torch
        >>>
        >>> from pytorch_widedeep.losses import ZILNLoss
        >>>
        >>> # REGRESSION
        >>> target = torch.tensor([[0., 1.5]]).view(-1, 1)
        >>> input = torch.tensor([[.1, .2, .3], [.4, .5, .6]])
        >>> ZILNLoss()(input, target)
        tensor([0.6287, 1.9941])
        """
        positive = target>0
        positive = positive.float()

        assert input.shape == torch.Size([target.shape[0], 3]), "Wrong shape of input."
        positive_input = input[..., :1]

        classification_loss = F.binary_cross_entropy_with_logits(positive_input, positive, reduction="none").flatten()        

        loc = input[..., 1:2]
        scale = torch.maximum(
            F.softplus(input[..., 2:]),
            torch.sqrt(torch.Tensor([torch.finfo(torch.float32).eps])))
        safe_labels = positive * target + (
            1 - positive) * torch.ones_like(target)

        regression_loss = -torch.mean(
            positive * torch.distributions.log_normal.LogNormal(loc=loc, scale=scale).log_prob(safe_labels),
            dim=-1)

        return torch.mean(classification_loss + regression_loss)

In [84]:
use_cuda = torch.cuda.is_available()
target = torch.tensor([[0., 1.5]]).view(-1, 1)
input = torch.tensor([[.1, .2, .3], [.4, .5, .6]])
ZILNLoss()(input, target)

tensor(1.3114)

# Keras implementation - original

* https://github.com/google/lifetime_value/blob/master/lifetime_value/zero_inflated_lognormal.py

In [67]:
import tensorflow.compat.v1 as tf
import tensorflow_probability as tfp
tfd = tfp.distributions


def zero_inflated_lognormal_pred(logits: tf.Tensor) -> tf.Tensor:
  """Calculates predicted mean of zero inflated lognormal logits.
  Arguments:
    logits: [batch_size, 3] tensor of logits.
  Returns:
    preds: [batch_size, 1] tensor of predicted mean.
  """
  logits = tf.convert_to_tensor(logits, dtype=tf.float32)
  positive_probs = tf.keras.backend.sigmoid(logits[..., :1])
  loc = logits[..., 1:2]
  scale = tf.keras.backend.softplus(logits[..., 2:])
  preds = (
      positive_probs *
      tf.keras.backend.exp(loc + 0.5 * tf.keras.backend.square(scale)))
  return preds


def zero_inflated_lognormal_loss(labels: tf.Tensor,
                                 logits: tf.Tensor) -> tf.Tensor:
  """Computes the zero inflated lognormal loss.
  Usage with tf.keras API:
  ```python
  model = tf.keras.Model(inputs, outputs)
  model.compile('sgd', loss=zero_inflated_lognormal)
  ```
  Arguments:
    labels: True targets, tensor of shape [batch_size, 1].
    logits: Logits of output layer, tensor of shape [batch_size, 3].
  Returns:
    Zero inflated lognormal loss value.
  """
  labels = tf.convert_to_tensor(labels, dtype=tf.float32)
  positive = tf.cast(labels > 0, tf.float32)

  logits = tf.convert_to_tensor(logits, dtype=tf.float32)
  logits.shape.assert_is_compatible_with(
      tf.TensorShape(labels.shape[:-1].as_list() + [3]))

  positive_logits = logits[..., :1]
  classification_loss = tf.keras.losses.binary_crossentropy(
      y_true=positive, y_pred=positive_logits, from_logits=True)

  loc = logits[..., 1:2]
  scale = tf.math.maximum(
      tf.keras.backend.softplus(logits[..., 2:]),
      tf.math.sqrt(tf.keras.backend.epsilon()))
  safe_labels = positive * labels + (
      1 - positive) * tf.keras.backend.ones_like(labels)
  regression_loss = -tf.keras.backend.mean(
      positive * tfd.LogNormal(loc=loc, scale=scale).log_prob(safe_labels),
      axis=-1)

  return classification_loss + regression_loss

In [68]:
target = tf.reshape(tf.constant([[0., 1.5]]), [-1, 1])
input = tf.constant([[.1, .2, .3], [.4, .5, .6]])
zero_inflated_lognormal_loss(target, input)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.7443967, 1.8783727], dtype=float32)>

In [69]:
zero_inflated_lognormal_pred(input)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.92363787],
       [1.6907552 ]], dtype=float32)>